Feature engineering

In [ ]:
# ==============================================================================
# --- CONFIGURATION AREA (Modify all adjustable parameters here) ---
# ==============================================================================

# --- 1. File Path Configuration ---
INPUT_FILE = 'Original database-FGA.xlsx'  # Input Excel filename
OUTPUT_DIR = 'feature_engineering_output' # Directory for all output results

# --- 2. Core Algorithm Parameters ---
RANDOM_STATE = 0  # Random seed for reproducibility (generally does not need modification)
PEARSON_CORR_THRESHOLD = 0.8  # Pearson correlation threshold for Stage 1. Features exceeding this will be processed.

# --- 2.1 Filter Method Selection Criterion ---
# In Stage 1, when two features are highly correlated, one must be eliminated.
# This parameter determines which one to retain.
# Options:
#   'mutual_info': Retain feature with higher [Mutual Information] with target (Recommended, captures non-linear relationships).
#   'pearson':     Retain feature with higher [Pearson Correlation] with target (Captures linear relationships only).
FILTER_METHOD_CRITERION = 'mutual_info'

# --- 3. RF (Random Forest) Model Hyperparameters ---
# Random Forest provides high robustness and resistance to overfitting, making it a stable choice for feature selection.
RF_N_ESTIMATORS = 500        # Number of trees. 500 is generally stable for various sample sizes.
RF_MAX_DEPTH = 4             # **Critical Parameter**: Strictly limit tree depth to prevent overfitting.
RF_MIN_SAMPLES_SPLIT = 8     # Minimum samples required to split an internal node (approx. 10%), avoids learning noise.
RF_MIN_SAMPLES_LEAF = 4      # Minimum samples required at a leaf node, ensuring predictions are based on a group of samples.
RF_MAX_FEATURES = 'sqrt'     # Number of features to consider when looking for the best split (square root), increases diversity.

# --- 4. SHAP Calculation & Iterative Screening Parameters ---
PERFORMANCE_METRIC = 'r2'  # Metric used during iterative screening. Options: 'mae' (Lower is better) or 'r2' (Higher is better).
SHAP_COARSE_SELECTION_PERCENT = 0.8  # Percentage of features to retain in SHAP coarse selection Stage. 0.8 means retaining top 80%.
EARLY_STOPPING_PATIENCE = 50 # Early stopping "patience". Stop iteration if performance does not improve for this many consecutive rounds.

# --- 5. Cross-Validation Parameters ---
KFOLD_SPLITS = 10 # Number of folds (k) for K-Fold Cross-Validation. Common values are 5 or 10.

# --- 6. Dataset Slicing Parameters ---
# FEATURE_COLUMN_SLICE: String defining the slice of feature columns. '1:-1' means from 2nd column to the second-to-last column.
FEATURE_COLUMN_SLICE = '1:-2'
# TARGET_COLUMN_INDEX: Integer index of the target column. -1 represents the last column.
TARGET_COLUMN_INDEX = -2
# TARGET_COLUMN_INDEX = -1


# ==============================================================================
# --- MAIN CODE BODY (Do not modify the following section) ---
# ==============================================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import shap
from tqdm import tqdm
import os
import time
from collections import defaultdict
import textwrap

def load_data_from_excel():
    """
    Load data from the Excel file, extract features (X) and target (Y) based on configuration,
    and perform data cleaning.
    """
    print(f"Loading data from '{INPUT_FILE}'...")
    if not os.path.exists(INPUT_FILE):
        print(f"Error: File '{INPUT_FILE}' does not exist.")
        exit()

    df = pd.read_excel(INPUT_FILE)
    df_original = df.copy()

    try:
        all_col_names = df.columns
        
        try:
            slice_parts = FEATURE_COLUMN_SLICE.split(':')
            start = int(slice_parts[0]) if slice_parts[0] else None
            end = int(slice_parts[1]) if len(slice_parts) > 1 and slice_parts[1] else None
            feature_slice = slice(start, end)
            
            feature_col_names = all_col_names[feature_slice]
            target_col_name = all_col_names[TARGET_COLUMN_INDEX]
        except (ValueError, IndexError) as e:
            print(f"Error: Invalid dataset slicing parameters '{FEATURE_COLUMN_SLICE}' or '{TARGET_COLUMN_INDEX}'. Please check configuration. Error details: {e}")
            exit()

        if len(feature_col_names) > 1:
            feature_range_str = f"From column '{feature_col_names[0]}' to '{feature_col_names[-1]}'"
        elif len(feature_col_names) == 1:
            feature_range_str = f"Column '{feature_col_names[0]}'"
        else:
            feature_range_str = "No features"

        print(f"Extracting features ({feature_range_str}) and target '{target_col_name}'.")

        if target_col_name in feature_col_names:
            print(f"Critical Error: Target column '{target_col_name}' is also identified as a feature column. Check Excel column order or slicing parameters.")
            exit()

        X = df[feature_col_names]
        Y = df[target_col_name]

    except Exception as e:
        print(f"Unknown error occurred during column extraction: {e}.")
        exit()

    print(f"Features X extracted, shape: {X.shape}")
    print(f"Target Y '{Y.name}' extracted, shape: {Y.shape}")

    combined_df = pd.concat([X, Y], axis=1)
    for col in combined_df.columns:
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')

    original_rows = len(combined_df)
    combined_df_cleaned = combined_df.dropna()

    if len(combined_df_cleaned) < original_rows:
        print(f"Warning: Removed {original_rows - len(combined_df_cleaned)} rows containing missing values.")
        df_original = df_original.loc[combined_df_cleaned.index]

    X_cleaned = combined_df_cleaned.drop(columns=[Y.name])
    y_cleaned = combined_df_cleaned[Y.name]

    print(f"Data loading complete. X shape: {X_cleaned.shape}, y shape: {y_cleaned.shape}")
    return X_cleaned, y_cleaned, df_original

def step1_filter_high_correlated_features(X, y):
    """
    Stage 1: Remove highly correlated features using filter methods.
    The selection criterion is determined by the FILTER_METHOD_CRITERION parameter.
    """
    print("\n--- Stage 1: Filter High-Correlation Features ---")

    if X.shape[1] <= 1:
        print("Insufficient feature count (<=1), skipping correlation filtering.")
        return X.copy(), []
    if X.shape[0] <= 1:
        print("Insufficient sample size (<=1), skipping correlation filtering.")
        return X.copy(), []

    corr_matrix = X.corr(method='pearson').abs()
    
    # --- Select filtering logic based on parameter ---
    if FILTER_METHOD_CRITERION == 'mutual_info':
        print("Criterion: Retain feature with higher [Mutual Information] with target in correlated pairs.")
        mi_scores = mutual_info_regression(X, y, random_state=RANDOM_STATE)
        feature_y_importance = pd.Series(mi_scores, index=X.columns)
        sorted_features = feature_y_importance.sort_values(ascending=False).index.tolist()
        criterion_name = "Mutual Information"
    elif FILTER_METHOD_CRITERION == 'pearson':
        print("Criterion: Retain feature with higher [Pearson Correlation] with target in correlated pairs.")
        feature_y_corr = {col: abs(X[col].corr(y)) for col in X.columns}
        feature_y_importance = pd.Series(feature_y_corr)
        sorted_features = sorted(X.columns, key=lambda col: feature_y_importance.get(col, -1), reverse=True)
        criterion_name = "Pearson Correlation"
    else:
        print(f"Error: Invalid criterion '{FILTER_METHOD_CRITERION}'. Please select 'pearson' or 'mutual_info' in the configuration area.")
        exit()

    kept_features_final = []
    all_dropped_features_set = set()
    retained_to_dropped_map = defaultdict(list)

    for current_feature in sorted_features:
        if current_feature in all_dropped_features_set:
            continue
        kept_features_final.append(current_feature)
        for other_feature in sorted_features:
            if other_feature == current_feature or other_feature in all_dropped_features_set:
                continue
            if corr_matrix.loc[current_feature, other_feature] > PEARSON_CORR_THRESHOLD:
                all_dropped_features_set.add(other_feature)
                retained_to_dropped_map[current_feature].append(other_feature)

    to_drop_list = list(all_dropped_features_set)

    if to_drop_list:
        print(f"Based on Pearson correlation (>{PEARSON_CORR_THRESHOLD}) and {criterion_name} with target, the following features are removed:")
        output_data = []
        for kept_feat, dropped_list in retained_to_dropped_map.items():
            if dropped_list:
                output_data.append({
                    'kept_feature': kept_feat,
                    'dropped_features': ", ".join(sorted(dropped_list))
                })
        if output_data:
            col1_header = f'Retained Feature (Higher {criterion_name})'
            max_kept_len = max(len(row['kept_feature']) for row in output_data)
            col1_width = max(max_kept_len, len(col1_header)) + 4
            col2_header = 'Eliminated Features (High correlation with retained)'
            terminal_width = 120
            col2_width = terminal_width - col1_width
            print(f"\n{col1_header:<{col1_width}}{col2_header}")
            print(f"{'-' * (col1_width - 1)} {'-' * len(col2_header)}")
            for row in output_data:
                kept_feat = row['kept_feature']
                dropped_feats_str = row['dropped_features']
                wrapped_lines = textwrap.wrap(dropped_feats_str, width=col2_width)
                print(f"{kept_feat:<{col1_width}}{wrapped_lines[0] if wrapped_lines else ''}")
                for i in range(1, len(wrapped_lines)):
                    print(f"{'':<{col1_width}}{wrapped_lines[i]}")
        X_filtered = X.drop(columns=to_drop_list)
    else:
        print("No feature pairs exceeded the Pearson correlation threshold. No features removed.")
        X_filtered = X.copy()

    print(f"\nFeature count after filtering: {X_filtered.shape[1]}")
    return X_filtered, to_drop_list


def step2_embed_shap_coarse_selection(X_filtered, y):
    """
    Stage 2: Coarse feature selection based on SHAP values (Embedded method).
    """
    print(f"\n--- Stage 2: SHAP Coarse Selection ({KFOLD_SPLITS}-Fold CV) ---")

    if X_filtered.shape[1] == 0:
        print("No features available for SHAP coarse selection.")
        return X_filtered.copy(), []
    if X_filtered.shape[0] < KFOLD_SPLITS:
        print(f"Sample size ({X_filtered.shape[0]}) is less than K-folds ({KFOLD_SPLITS}). Cannot perform Cross-Validation.")
        return X_filtered.copy(), []

    kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    avg_abs_shap_values_per_fold = []

    print(f"Calculating SHAP values (Using {KFOLD_SPLITS}-Fold CV)...")
    
    # Check if sample size meets model minimum requirements
    train_size_per_fold = X_filtered.shape[0] * (KFOLD_SPLITS - 1) // KFOLD_SPLITS
    min_samples_for_model = max(RF_MIN_SAMPLES_LEAF, RF_MIN_SAMPLES_SPLIT)
    if train_size_per_fold < min_samples_for_model:
        print(f"Warning: Training set size per fold ({train_size_per_fold}) is smaller than RF model minimum requirement ({min_samples_for_model}). Skipping SHAP coarse selection.")
        return X_filtered.copy(), []

    for train_idx, val_idx in tqdm(kf.split(X_filtered), total=KFOLD_SPLITS, desc="SHAP Coarse Selection Progress"):
        X_fold_train, y_fold_train = X_filtered.iloc[train_idx], y.iloc[train_idx]
        scaler = StandardScaler().fit(X_fold_train)
        X_fold_train_scaled = scaler.transform(X_fold_train)
        
        rf_model = RandomForestRegressor(
            n_estimators=RF_N_ESTIMATORS,
            max_depth=RF_MAX_DEPTH,
            min_samples_split=RF_MIN_SAMPLES_SPLIT,
            min_samples_leaf=RF_MIN_SAMPLES_LEAF,
            max_features=RF_MAX_FEATURES,
            random_state=RANDOM_STATE,
            n_jobs=-1
        )
        try:
            rf_model.fit(X_fold_train_scaled, y_fold_train)
            explainer = shap.TreeExplainer(rf_model)
            shap_values_fold = explainer.shap_values(X_fold_train_scaled)
            if shap_values_fold.ndim == 1:
                avg_abs_shap_values_per_fold.append(np.abs(shap_values_fold))
            else:
                avg_abs_shap_values_per_fold.append(np.abs(shap_values_fold).mean(axis=0))
        except Exception as e:
            print(f"\nWarning: Model training or SHAP calculation failed during coarse selection: {e}")
            continue

    if not avg_abs_shap_values_per_fold:
        print("Warning: Failed to calculate any SHAP values. Skipping SHAP coarse selection.")
        return X_filtered.copy(), []

    feature_importances_shap = pd.Series(np.mean(avg_abs_shap_values_per_fold, axis=0), index=X_filtered.columns).sort_values(ascending=False)
    print("\nFeature Importance (Average Absolute SHAP values across K-Folds):")
    print(feature_importances_shap)

    num_features_to_keep = int(len(feature_importances_shap) * SHAP_COARSE_SELECTION_PERCENT)
    if num_features_to_keep == 0 and len(feature_importances_shap) > 0: num_features_to_keep = 1
    if num_features_to_keep > len(feature_importances_shap): num_features_to_keep = len(feature_importances_shap)

    features_to_keep = feature_importances_shap.index[:num_features_to_keep].tolist()
    features_to_drop = list(set(X_filtered.columns) - set(features_to_keep))

    if features_to_drop:
        print(f"\nBased on SHAP Coarse Selection (Retaining top {SHAP_COARSE_SELECTION_PERCENT * 100:.1f}%), the following features are removed: {features_to_drop}")
        X_shap_coarse = X_filtered[features_to_keep]
    else:
        print("\nNo features removed after SHAP coarse selection.")
        X_shap_coarse = X_filtered.copy()

    print(f"Feature count after SHAP coarse selection: {X_shap_coarse.shape[1]}")
    return X_shap_coarse, features_to_drop


def step3_wrapper_shap_iterative_selection(X_shap_coarse, y):
    """
    Stage 3: Iterative feature refinement using Wrapper method with early stopping mechanism.
    """
    print(f"\n--- Stage 3: Iterative Feature Refinement ({KFOLD_SPLITS}-Fold CV) ---")

    if X_shap_coarse.shape[1] <= 1:
        print("Insufficient feature count (<=1), cannot perform iterative refinement.")
        return list(X_shap_coarse.columns), []
    if X_shap_coarse.shape[0] < KFOLD_SPLITS:
        print(f"Sample size ({X_shap_coarse.shape[0]}) is less than K-folds ({KFOLD_SPLITS}). Cannot perform Cross-Validation.")
        return list(X_shap_coarse.columns), []

    current_features = list(X_shap_coarse.columns)
    best_features = list(current_features)

    if PERFORMANCE_METRIC == 'r2':
        best_score = -np.inf
        is_better = lambda current, best: current > best
    elif PERFORMANCE_METRIC == 'mae':
        best_score = np.inf
        is_better = lambda current, best: current < best
    else:
        raise ValueError("Unsupported performance metric. Please select 'r2' or 'mae'.")

    performance_history = []
    kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=RANDOM_STATE)

    # Check if sample size meets model minimum requirements
    train_size_per_fold = X_shap_coarse.shape[0] * (KFOLD_SPLITS - 1) // KFOLD_SPLITS
    min_samples_for_model = max(RF_MIN_SAMPLES_LEAF, RF_MIN_SAMPLES_SPLIT)
    if train_size_per_fold < min_samples_for_model:
        print(f"Warning: Training set size per fold ({train_size_per_fold}) is smaller than RF model minimum requirement ({min_samples_for_model}). Skipping iterative refinement.")
        return list(X_shap_coarse.columns), []

    print("Calculating baseline performance for initial feature set...")
    initial_fold_scores = []
    for train_idx, val_idx in tqdm(kf.split(X_shap_coarse), total=KFOLD_SPLITS, desc="Baseline Evaluation Progress"):
        X_fold_train, y_fold_train = X_shap_coarse.iloc[train_idx][current_features], y.iloc[train_idx]
        X_fold_val, y_fold_val = X_shap_coarse.iloc[val_idx][current_features], y.iloc[val_idx]
        scaler_fold = StandardScaler().fit(X_fold_train)
        X_fold_train_scaled = scaler_fold.transform(X_fold_train)
        X_fold_val_scaled = scaler_fold.transform(X_fold_val)

        rf_model = RandomForestRegressor(
            n_estimators=RF_N_ESTIMATORS,
            max_depth=RF_MAX_DEPTH,
            min_samples_split=RF_MIN_SAMPLES_SPLIT,
            min_samples_leaf=RF_MIN_SAMPLES_LEAF,
            max_features=RF_MAX_FEATURES,
            random_state=RANDOM_STATE,
            n_jobs=-1
        )
        try:
            rf_model.fit(X_fold_train_scaled, y_fold_train)
            y_pred = rf_model.predict(X_fold_val_scaled)
            if PERFORMANCE_METRIC == 'r2':
                initial_fold_scores.append(r2_score(y_fold_val, y_pred))
            elif PERFORMANCE_METRIC == 'mae':
                initial_fold_scores.append(mean_absolute_error(y_fold_val, y_pred))
        except Exception as e:
            print(f"\nWarning: Model training or prediction failed during baseline evaluation: {e}")
            continue

    if not initial_fold_scores:
        print("Warning: Initial feature set failed to produce any valid scores. Cannot perform iterative refinement.")
        return list(X_shap_coarse.columns), []

    best_score = np.mean(initial_fold_scores)
    performance_history.append({'num_features': len(current_features), PERFORMANCE_METRIC: best_score})
    print(f"  Initial Feature Set ({len(current_features)} features), Avg {PERFORMANCE_METRIC.upper()}: {best_score:.4f}")

    print("Starting iterative feature removal...")
    
    non_improvement_streak = 0
    
    while len(current_features) > 1:
        current_iteration_shap_values_for_drop = []
        for train_idx, val_idx in kf.split(X_shap_coarse):
            X_fold_train, y_fold_train = X_shap_coarse.iloc[train_idx][current_features], y.iloc[train_idx]
            scaler_fold = StandardScaler().fit(X_fold_train)
            X_fold_train_scaled = scaler_fold.transform(X_fold_train)
            
            rf_model = RandomForestRegressor(
                n_estimators=RF_N_ESTIMATORS,
                max_depth=RF_MAX_DEPTH,
                min_samples_split=RF_MIN_SAMPLES_SPLIT,
                min_samples_leaf=RF_MIN_SAMPLES_LEAF,
                max_features=RF_MAX_FEATURES,
                random_state=RANDOM_STATE,
                n_jobs=-1
            )
            try:
                rf_model.fit(X_fold_train_scaled, y_fold_train)
                explainer = shap.TreeExplainer(rf_model)
                shap_values_fold = explainer.shap_values(X_fold_train_scaled)
                if shap_values_fold.ndim == 1:
                    current_iteration_shap_values_for_drop.append(np.abs(shap_values_fold))
                else:
                    current_iteration_shap_values_for_drop.append(np.abs(shap_values_fold).mean(axis=0))
            except Exception:
                continue

        if not current_iteration_shap_values_for_drop:
            print(f"  Warning: Iteration failed, unable to calculate SHAP values. Stopping iteration.")
            break

        avg_iteration_shap = np.mean(current_iteration_shap_values_for_drop, axis=0)
        feature_importances_current = pd.Series(avg_iteration_shap, index=current_features).sort_values(ascending=True)
        feature_to_drop = feature_importances_current.index[0]
        current_features.remove(feature_to_drop)
        print(f"  Removing least important feature: {feature_to_drop}")

        fold_scores_after_drop = []
        for train_idx, val_idx in tqdm(kf.split(X_shap_coarse), total=KFOLD_SPLITS, desc=f"Evaluating after removal"):
            X_fold_train, y_fold_train = X_shap_coarse.iloc[train_idx][current_features], y.iloc[train_idx]
            X_fold_val, y_fold_val = X_shap_coarse.iloc[val_idx][current_features], y.iloc[val_idx]
            scaler_fold = StandardScaler().fit(X_fold_train)
            X_fold_train_scaled = scaler_fold.transform(X_fold_train)
            X_fold_val_scaled = scaler_fold.transform(X_fold_val)
            
            rf_model = RandomForestRegressor(
                n_estimators=RF_N_ESTIMATORS,
                max_depth=RF_MAX_DEPTH,
                min_samples_split=RF_MIN_SAMPLES_SPLIT,
                min_samples_leaf=RF_MIN_SAMPLES_LEAF,
                max_features=RF_MAX_FEATURES,
                random_state=RANDOM_STATE,
                n_jobs=-1
            )
            try:
                rf_model.fit(X_fold_train_scaled, y_fold_train)
                y_pred = rf_model.predict(X_fold_val_scaled)
                if PERFORMANCE_METRIC == 'r2':
                    fold_scores_after_drop.append(r2_score(y_fold_val, y_pred))
                elif PERFORMANCE_METRIC == 'mae':
                    fold_scores_after_drop.append(mean_absolute_error(y_fold_val, y_pred))
            except Exception:
                continue

        if not fold_scores_after_drop:
            print(f"  Warning: Unable to calculate any valid scores after feature removal. Stopping iteration.")
            break

        current_avg_score = np.mean(fold_scores_after_drop)
        performance_history.append({'num_features': len(current_features), PERFORMANCE_METRIC: current_avg_score})
        print(f"  Feature Count: {len(current_features)}, Avg {PERFORMANCE_METRIC.upper()}: {current_avg_score:.4f}")

        if is_better(current_avg_score, best_score):
            best_score, best_features = current_avg_score, list(current_features)
            print(f"    -> Performance improved, resetting patience. Current best feature set size: {len(best_features)}")
            non_improvement_streak = 0
        else:
            non_improvement_streak += 1
            print(f"    -> No improvement (Streak {non_improvement_streak}/{EARLY_STOPPING_PATIENCE})")
            if non_improvement_streak >= EARLY_STOPPING_PATIENCE:
                print(f"    -> Performance has not improved for {EARLY_STOPPING_PATIENCE} consecutive rounds. Triggering early stopping.")
                break

    print(f"\nIterative refinement completed. Best feature set ({len(best_features)} features): {best_features}")
    return best_features, performance_history


if __name__ == "__main__":
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    if not os.path.exists(INPUT_FILE):
        print(f"File '{INPUT_FILE}' not found. Creating dummy data...")
        num_samples, feature_cols = 70, 162
        df_list = [pd.DataFrame([f'Sample_{i+1}' for i in range(num_samples)], columns=['SampleID'])]
        all_feature_names = [f'F{i+1}' for i in range(feature_cols)]
        feature_data = np.random.rand(num_samples, len(all_feature_names))
        df_features = pd.DataFrame(feature_data, columns=all_feature_names)
        df_list.append(df_features)
        df_dummy = pd.concat(df_list, axis=1)
        y_values = 5 * df_dummy['F1'] + 3 * df_dummy['F2']**2 + np.random.randn(num_samples) * 0.5
        df_dummy['Target'] = y_values
        df_dummy.to_excel(INPUT_FILE, index=False)
        print(f"Dummy data created: '{INPUT_FILE}'.")

    X_full, y_full, df_original = load_data_from_excel()
    if not X_full.empty:
        original_corr_matrix = X_full.corr(method='pearson')
        original_corr_path = os.path.join(OUTPUT_DIR, f'Original_Feature_Correlation_Matrix_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')
        original_corr_matrix.to_excel(original_corr_path, index=True)
        print(f"Saved: Original feature correlation matrix -> {original_corr_path}")

    X_filtered, dropped_pearson_list = step1_filter_high_correlated_features(X_full, y_full)
    if not X_filtered.empty and X_filtered.shape[1] > 1:
        filtered_corr_matrix = X_filtered.corr(method='pearson')
        filtered_corr_path = os.path.join(OUTPUT_DIR, f'Stage1_Filtered_Feature_Correlation_Matrix_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')
        filtered_corr_matrix.to_excel(filtered_corr_path, index=True)
        print(f"Saved: Stage 1 filtered feature correlation matrix -> {filtered_corr_path}")

    X_shap_coarse, dropped_shap = step2_embed_shap_coarse_selection(X_filtered, y_full)
    final_features, performance_history = step3_wrapper_shap_iterative_selection(X_shap_coarse, y_full)

    if final_features:
        print("\n--- Sorting final features by original order ---")
        original_feature_order = X_full.columns
        sorted_final_features = [col for col in original_feature_order if col in final_features]
        final_features = sorted_final_features
        print(f"Final features sorted according to original column order.")

    print(f"\n--- Exporting final results to '{OUTPUT_DIR}' directory ---")
    num_dropped_pearson = len(dropped_pearson_list)
    summary_df = pd.DataFrame({
        'Stage': ['Original Features', 'Stage 1: Pearson Correlation Filter', 'Stage 2: SHAP Coarse Selection', 'Stage 3: SHAP Iterative Refinement'],
        'Feature Count': [X_full.shape[1], X_filtered.shape[1], X_shap_coarse.shape[1], len(final_features)],
        'Features Removed': [0, num_dropped_pearson, len(dropped_shap), X_shap_coarse.shape[1] - len(final_features)]
    })
    summary_path = os.path.join(OUTPUT_DIR, f'Feature_Engineering_Process_Summary_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')
    summary_df.to_excel(summary_path, index=False)
    print(f"Saved: Feature engineering process summary -> {summary_path}")

    history_path = os.path.join(OUTPUT_DIR, f'Performance_Iteration_History_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')
    pd.DataFrame(performance_history).to_excel(history_path, index=False)
    print(f"Saved: Performance iteration history -> {history_path}")

    if len(final_features) > 1:
        final_features_df = df_original[final_features]
        final_corr_matrix = final_features_df.corr(method='pearson')
        final_corr_path = os.path.join(OUTPUT_DIR, f'Final_Feature_Correlation_Matrix_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')
        final_corr_matrix.to_excel(final_corr_path, index=True)
        print(f"Saved: Final feature correlation matrix -> {final_corr_path}")

    if final_features:
        cols_to_export = []
        if not df_original.empty and df_original.columns[0] not in final_features and df_original.columns[0] != y_full.name:
                cols_to_export.append(df_original.columns[0])
        
        cols_to_export.extend(final_features)
        if y_full.name in df_original.columns:
            cols_to_export.append(y_full.name)
        
        unique_cols_to_export = list(dict.fromkeys(cols_to_export))

        dataset_path = os.path.join(OUTPUT_DIR, f'Final_Selected_Dataset_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')
        df_original[unique_cols_to_export].to_excel(dataset_path, index=False)
        print(f"Saved: Final selected dataset -> {dataset_path}")
    else:
        print("No final features selected. Final dataset not saved.")

    print("\nAll processing completed.")